In [1]:
import numpy as np
from datasets import load_all_data_mnist, load_all_data_cifar10, load_all_data_fashion_mnist, load_all_data_apply_vgg_cifar10
from utils import create_batch_data
import modules
import layers
from criterions import FiniteFieldMSELoss

In [2]:
BATCH_SIZE = 256
EPOCH = 5
PRINT = 10
FLATTEN = False
# 0, MNIST; 1, FashionMNIST; 2, CIFAR10; 3, VGG-CIFAR10
DATASET_MODE = 2
PRIME = 2**26 - 5
QUANTIZATION_WEIGHT = 8
QUANTIZATION_INPUT = 8
QUANTIZATION_BATCH_SIZE = 8
LR = 7

In [ ]:
# data fetching
load_path = '../../data'
if DATASET_MODE == 0:
    train_data, train_label, test_data, test_label = load_all_data_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, flatten=FLATTEN)
elif DATASET_MODE == 1:
    train_data, train_label, test_data, test_label = load_all_data_fashion_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, flatten=FLATTEN)
elif DATASET_MODE == 2:
    train_data, train_label, test_data, test_label = load_all_data_cifar10(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, flatten=FLATTEN)
elif DATASET_MODE == 3:
    train_data, train_label, test_data, test_label = load_all_data_apply_vgg_cifar10(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, flatten=FLATTEN)
else:
    train_data, train_label, test_data, test_label = None, None, None, None
train_data, train_label, test_data, test_label = create_batch_data(train_data, train_label, test_data, test_label, BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model_arr = [
    layers.FiniteFieldPiNetSecondOrderConvLayer(3, 6, (9, 9), QUANTIZATION_WEIGHT, PRIME, first_layer=True, quantization_bit_input=QUANTIZATION_INPUT),
    modules.Flatten(),
    layers.FiniteFieldPiNetSecondOrderLinearLayer(3456, 128, QUANTIZATION_WEIGHT, PRIME),
    layers.FiniteFieldLinearLayer(128, 10, QUANTIZATION_WEIGHT, PRIME)
]

model = modules.Network(model_arr)
criterion = FiniteFieldMSELoss(PRIME, QUANTIZATION_WEIGHT, QUANTIZATION_BATCH_SIZE)

In [ ]:
for epoch in range(EPOCH):
    tot_loss = 0
    for train_idx, (train_data_batch, train_label_batch) in enumerate(zip(train_data, train_label)):
        # train
        preds = model.forward(train_data_batch)

        tot_loss += criterion.forward(preds, train_label_batch)
        propagated_error = criterion.error_derivative()

        model.backprop(propagated_error)
        model.optimize(LR)

        if train_idx == 0 or (train_idx + 1) % PRINT == 0:
            tot_acc = 0
            tot_sample = 0
            for train_acc_idx, (test_data_batch, test_label_batch) in enumerate(zip(test_data, test_label)):
                # train accuracy
                preds = model.forward(test_data_batch)
                pred_args = np.argmax(preds, axis=1)

                tot_acc += np.count_nonzero(pred_args == test_label_batch)
                tot_sample += test_data_batch.shape[0]
            accuracy = tot_acc / tot_sample
            if train_idx != 0:
                tot_loss = tot_loss / PRINT
            print('epoch: {}, idx: {}, accuracy: {}, loss: {}'.format(epoch + 1, train_idx + 1, accuracy, tot_loss))
            tot_loss = 0

epoch: 1, idx: 1, accuracy: 0.1269, loss: 1.3846830877451854
epoch: 1, idx: 10, accuracy: 0.2125, loss: 0.955424733710205
epoch: 1, idx: 20, accuracy: 0.2505, loss: 0.9470299719219636
epoch: 1, idx: 30, accuracy: 0.273, loss: 0.9001624456508196
epoch: 1, idx: 40, accuracy: 0.2853, loss: 0.8955623976716316
epoch: 1, idx: 50, accuracy: 0.3061, loss: 0.8954912565571391
